In [1]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd
import mysql
import time
import datetime
import configparser
import pymysql

In [2]:
def extract_currency_table(url):
    content = requests.get(url).text 
    soup= BeautifulSoup(content,'lxml')
    table_content = soup.find('div', class_='table_conv')
    return table_content 

In [3]:
def extract_currency_data(table_content):
    data ={}
    theaders = table_content.find_all('th')
    for header in theaders:
        data[header.text]=[]
    headers= list(data.keys())
    #print("The table headers are: ", headers)

    trows = table_content.find_all('tr')
    for row in trows: 
            row_data_list = row.find_all('td')
            for index in range(len(row_data_list)):
                 data[headers[index]].append(row_data_list[index].text)
    #print(data)
    return data 


In [4]:
#saving the data into a csv file
#df.to_csv(r'database.csv', index = False)

In [5]:
#insert your data base connection info 
def create_database():
    return  pymysql.connect(host='localhost',user='root',password='Zeineb2001///',db='data_base')

In [6]:
#db = create_database()
def create_table(db):
    cursor = db.cursor()
    sql = """ CREATE TABLE IF NOT EXISTS data (
    Désignation CHAR(60),
    Code CHAR(3),
    Unité INT,
    Achat  FLOAT, 
    Vente FLOAT, 
    Date DATE
    )
    """
    cursor.execute(sql)
    db.commit()
    cursor.close()

In [7]:
def insert_data (db,df):
    cursor = db.cursor()
    time_stamp = datetime.datetime.now()
    Date = time_stamp.strftime("%Y-%m-%d")
    print(Date)
    for index, row in df.iterrows():
        query = "INSERT INTO data ( Désignation, Code, Unité, Achat ,Vente, Date) VALUES (%s,%s, %s, %s, %s ,%s)"
        series= pd.Series ([Date])
        row=row.append(series)
        print(row)
        cursor.execute(query,tuple(row.values))
    db.commit()
    cursor.close()    

In [8]:
#if db.open:
 #   print('yes')

In [ ]:
url = 'https://www.biat.tn/biat/Fr/cours-de-change_66_127'
db= create_database()
create_table(db)
for a in range(2):
    table_content= extract_currency_table(url)
    data=extract_currency_data(table_content)
    df = pd.DataFrame(data)
    insert_data (db,df) 
    #define the amount of time you want your program to sleep in seconds, here 86400s = 24H 
    time.sleep(60*60*24)
db.close()      

2022-09-25
Désignation    Ryal Saoudien
Code                     SAR
Unité                     10
Achat                  8.521
Vente                  8.869
0                 2022-09-25
dtype: object
Désignation    Dollar Canadien
Code                       CAD
Unité                        1
Achat                    2.349
Vente                    2.447
0                   2022-09-25
dtype: object
Désignation    Couronne Danoise
Code                        DKK
Unité                       100
Achat                    41.789
Vente                    43.504
0                    2022-09-25
dtype: object
Désignation    Dirham Emirats Arabe Unis
Code                                 AED
Unité                                 10
Achat                              8.641
Vente                              8.997
0                             2022-09-25
dtype: object
Désignation    Dollar des Etats-Unis
Code                             USD
Unité                              1
Achat                   